# Neophilic Gardener - Analysis 2

## Research question
Does gardening experience buffer the negative association between **food neophobia** and (1) **knowledge** and (2) **consumption** of neglected foods?

## Hypotheses
- **H1 (Cognitive)**: The negative association between food neophobia and knowledge is **weaker** among individuals with **higher gardening experience**.
- **H2 (Behavioral)**: The negative association between food neophobia and consumption is **weaker** among individuals with **higher gardening experience**.

## Model structure (moderation)
For each outcome (Y), we estimate:

$$ Y = b_0 + b_1(Neophobia) + b_2(Experience) + b_3(Neophobia \times Experience) + \varepsilon $$

Where the interaction term **b3** captures the *buffering / protective* effect.

## Notes
- The knowledge outcome is a **PCA(1)** composite of recognition, classification, and recall (species_total).
- The consumption outcome is an **annualized** + **log1p** + **0–1 normalized** index averaged across plants/mushrooms/algae.


In [1]:
# Core imports
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from IPython.display import display
from scipy.stats import chi2_contingency
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore")

# Visualization defaults
pio.templates.default = "plotly_white"
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 80)
pd.options.plotting.backend = "plotly"

# Output directory for figures
images_path = Path("../data/final_outputs/new_analysis_2")
images_path.mkdir(parents=True, exist_ok=True)
print(f"Figures will be saved to: {images_path.resolve()}")


Figures will be saved to: /media/nas-elias/pesquisas/papers/neophilic-gardener/data/final_outputs/new_analysis_2


## Data loading
We use the cleaned dataset produced by the data-cleaning notebook (`data/processed/data_cleaned.json`).

In [2]:
df_raw = pd.read_json("../data/processed/data_cleaned.json", lines=True)
print("Dataset shape:", df_raw.shape)
display(df_raw.head(3))


Dataset shape: (157, 89)


,id,date_birth,zip_code,urbanization_degree,gender,education_level,education_field,gardener,garden_type_communitary,garden_type_pedagogical,garden_type_other,garden_type_other_description,years_experience,frequency_experience,experience_description,g03q11,g03q12,g03q13,g03q14,g03q15,g03q16,g03q17,g03q18,g03q19,g03q20,recognition_grade,g03q21,g03q22,g03q23,g03q24,g03q25,g03q26,g03q27,g03q28,g03q29,g03q30,classification_grade,recall_plants,plants_valid,plants_total,recall_algae,algae_valid,algae_total,recall_mushrooms,mushrooms_valid,mushrooms_total,species_total,diet_type,consumption_plants_time_scale,consumption_plants_frequency,consumption_mushrooms_time_scale,consumption_mushrooms_frequency,consumption_algae_time_scale,consumption_algae_frequency,social_model_plants,social_model_mushrooms,social_model_algae,social_model_other,social_trigger_plants,social_trigger_mushrooms,social_trigger_algae,meals_friends,meals_family_older,meals_family_same_generation,meals_not_family_older,barrier_plant,barrier_mushroom,barrier_algae,barrier_other,g06q39,g06q40,g06q41,g06q42,g06q43,g06q44,g06q45,g06q46,g06q47,g06q48,g06q49,g06q50,g06q51,g06q52,extra_info,authorization_new_contact,interview_time,age,state,region
0,15,1986,59082110,Zona urbana,Mulher cis - pessoa que nasceu com sexo femini...,Mestrado concluído,Ciências Biológicas,Sim,Não,Sim,Não,NaN,0.0,1.0,"Participei de algumas oficinas, mas não tive c...",Nori,Não sei,Não sei,Shitake,Portobello,Wakame,Kombu,Taioba,Shimeji,Mangaba,8,Alga,Não sei,Não sei,Cogumelo,Cogumelo,Alga,Alga,"Planta - incluindo frutas, verduras e legumes",Cogumelo,"Planta - incluindo frutas, verduras e legumes",8,"rambutan, cupuaçu, mangostim, pitaya, almeirão...","rambutan, cupuaçu, mangostim, pitaya, almeirão...",6,As algas que conheço já foram mencionadas nest...,0,0.0,Os cogumelos que conheço já foram mencionados ...,0,0,6,Onívora - Possuo uma alimentação diversificada...,Nenhum,0,Mês,1,Mês,2,Não consumi ou consumo esse tipo de alimento,Familiares da mesma idade ou mais novos do que eu,Familiares da mesma idade ou mais novos do que eu,NaN,Familiares da mesma idade ou mais novos do que eu,Familiares da mesma idade ou mais novos do que eu,Familiares da mesma idade ou mais novos do que eu,0,0,16,0,Falta de conhecimento: não conheço esses alime...,Falta de acesso econômico: acho esse tipo de a...,Falta de acesso econômico: acho esse tipo de a...,NaN,1 - Discordo totalmente,1 - Discordo totalmente,1 - Discordo totalmente,1 - Discordo totalmente,6 - Concordo bastante,5 - Concordo pouco,1 - Discordo totalmente,1 - Discordo totalmente,7 - Concordo totalmente,5 - Concordo pouco,NaN,NaN,NaN,NaN,Me considero uma pessoa muito interessada em n...,"Sim, podem me contatar!",1432.75,39,RN,Northeast
1,17,1997,59090324,Zona urbana,Homem cis - pessoa que nasceu com sexo masculi...,Mestrado concluído,Nutrição,Sim,Sim,Sim,Não,NaN,7.0,5.0,"Participei de atividades de ensino, pesquisa e...",Nori,Beldroega,Tucumã,Shitake,Portobello,Wakame,Kombu,Taioba,Shimeji,Mangaba,10,Alga,"Planta - incluindo frutas, verduras e legumes","Planta - incluindo frutas, verduras e legumes",Cogumelo,"Planta - incluindo frutas, verduras e legumes",Alga,Alga,"Planta - incluindo frutas, verduras e legumes",Cogumelo,"Planta - incluindo frutas, verduras e legumes",9,"Urucum, taioba, ora-pro-nobis,fruta pão,jambú,...","urucum, ora-pro-nóbis, fruta-pão, jambu, cubiu...",14,Alga nori,0,0.0,"Shitake,shimeji, portubelo, champignon",0,0,14,Flexitariano - Tenho um cardápio flexível e co...,Semana,1,Mês,1,Mês,1,Professores ou pessoas de referência mais velh...,Familiares da mesma idade ou mais novos do que eu,Não consumi ou consumo esse tipo de alimento,NaN,Familiares da mesma idade ou mais novos do que eu,Professores ou pessoas de referência mais velh...,Meus amigos ou colegas de trabalho,3,2,0,2,Falta de acesso físico: não é fácil de achar e...,Outro,Outro,NaN,7 - Concordo totalmente,1 - Discordo totalmente,7 - Concordo totalmente,7 - Concordo totalmente,1 - Discordo

## Helper functions
These utility functions keep the analysis reproducible and readable.

In [3]:
def parse_likert_1_to_7(series: pd.Series) -> pd.Series:
    """Parse 1–7 Likert strings (e.g., '1 - Discordo totalmente') into integers.

    Args:
        series: Raw survey responses stored as strings.

    Returns:
        A numeric series with values in [1, 7] and NaN for non-parsable values.
    """
    extracted = series.astype(str).str.extract(r"^(?P<n>\d+)", expand=True)
    return pd.to_numeric(extracted["n"], errors="coerce")


def reverse_1_to_7(series: pd.Series) -> pd.Series:
    """Reverse-code a 1–7 Likert scale: 1↔7, 2↔6, 3↔5, 4↔4.

    Args:
        series: Numeric series on a 1–7 Likert scale.

    Returns:
        Reverse-coded numeric series.
    """
    return 8 - series


def safe_normalize_0_1(series: pd.Series) -> pd.Series:
    """Normalize a non-negative series to [0, 1] using max scaling.

    Args:
        series: Numeric series.

    Returns:
        A series scaled to [0, 1]. If max is 0, returns 0 for all non-missing values.
    """
    # Force numeric dtype; avoids statsmodels treating the regressor as categorical.
    s = pd.to_numeric(series, errors="coerce")
    max_val = s.max()
    if pd.isna(max_val) or max_val == 0:
        return s.fillna(0) * 0
    return s / max_val


def minmax_scale_0_100(series: pd.Series) -> pd.Series:
    """Min-max scale a series to [0, 100].

    Args:
        series: Numeric series.

    Returns:
        Series mapped to [0, 100]. If constant, returns 50 for non-missing values.
    """
    s = pd.to_numeric(series, errors="coerce")
    s_min = s.min()
    s_max = s.max()
    if pd.isna(s_min) or pd.isna(s_max) or s_max == s_min:
        return s.where(s.isna(), 50.0)
    return (s - s_min) / (s_max - s_min) * 100


def make_equal_frequency_bins_with_zero_as_none(series: pd.Series, n_bins_nonzero: int = 4) -> tuple[pd.Series, dict]:
    """Create experience bins: 'none' for zeros + equal-frequency bins for non-zero values.

    Args:
        series: Numeric series where 0 means no experience.
        n_bins_nonzero: Number of equal-frequency bins for non-zero values.

    Returns:
        A tuple of:
            - binned series with labels: none, bin_1..bin_k, missing
            - dict with bin ranges for non-zero bins
    """
    valid_mask = series.notna()
    zero_mask = (series == 0) & valid_mask
    non_zero_mask = (series > 0) & valid_mask

    binned = pd.Series(index=series.index, dtype="object")
    binned.loc[~valid_mask] = "missing"
    binned.loc[zero_mask] = "none"

    non_zero = series.loc[non_zero_mask].sort_values()
    bin_ranges: dict[str, dict] = {}

    if len(non_zero) == 0:
        return binned, bin_ranges

    n_samples = len(non_zero)
    samples_per_bin = n_samples // n_bins_nonzero
    remainder = n_samples % n_bins_nonzero

    start = 0
    for bin_num in range(1, n_bins_nonzero + 1):
        current_size = samples_per_bin + (1 if bin_num <= remainder else 0)
        end = start + current_size

        idx = non_zero.index[start:end]
        vals = non_zero.iloc[start:end]
        label = f"bin_{bin_num}"

        binned.loc[idx] = label
        bin_ranges[label] = {"min": float(vals.min()), "max": float(vals.max()), "count": int(current_size)}

        start = end

    return binned, bin_ranges


def cramers_v_from_crosstab(crosstab: pd.DataFrame) -> float:
    """Compute Cramér's V effect size from a contingency table.

    Args:
        crosstab: Contingency table (rows = groups, columns = categories).

    Returns:
        Cramér's V in [0, 1].
    """
    chi2, _, _, _ = chi2_contingency(crosstab)
    n = crosstab.to_numpy().sum()
    if n == 0:
        return np.nan
    r, k = crosstab.shape
    denom = min(r - 1, k - 1)
    if denom <= 0:
        return np.nan
    return float(np.sqrt((chi2 / n) / denom))


def annualize_consumption(time_scale: pd.Series, frequency: pd.Series) -> pd.Series:
    """Convert consumption frequency to an annualized rate.

    Args:
        time_scale: Series with categories such as 'Nenhum', 'Ano', 'Mês', 'Semana', 'Dia'.
        frequency: Series with numeric frequency within the time scale.

    Returns:
        A numeric series representing estimated events per year.
    """
    scale_map = {"Nenhum": 0, "Ano": 1, "Mês": 12, "Semana": 52, "Dia": 365}
    mult = time_scale.map(scale_map)
    f = pd.to_numeric(frequency, errors="coerce")
    return pd.to_numeric(mult, errors="coerce") * f


def zscore(series: pd.Series) -> pd.Series:
    """Z-score a numeric series.

    Args:
        series: Numeric series.

    Returns:
        Standardized series with mean 0 and std 1 (NaN-safe).
    """
    s = pd.to_numeric(series, errors="coerce")
    std = s.std(ddof=0)
    if pd.isna(std) or std == 0:
        return s * 0
    return (s - s.mean()) / std


def pca_1_component_index(df_in: pd.DataFrame, cols: list[str]) -> tuple[pd.Series, dict]:
    """Compute a 1-component PCA score (SVD) from the given columns.

    This avoids adding new dependencies (e.g., sklearn) while producing a robust
    composite score. We z-score inputs prior to SVD.

    Args:
        df_in: Input dataframe.
        cols: Columns to include in PCA.

    Returns:
        Tuple (scores, metadata)
          - scores: Series of PC1 scores (NaN for rows with missing inputs)
          - metadata: dict with explained variance ratio and loadings
    """
    X = df_in[cols].apply(pd.to_numeric, errors="coerce")
    z = (X - X.mean()) / X.std(ddof=0)

    # Only compute PCA on complete cases for stability
    mask = z.notna().all(axis=1)
    Z = z.loc[mask].to_numpy()

    if Z.shape[0] < 3:
        scores = pd.Series(index=df_in.index, dtype=float)
        meta = {"explained_variance_ratio": np.nan, "loadings": None}
        return scores, meta

    # Center across rows to match PCA conventions on standardized data
    Z = Z - Z.mean(axis=0)
    U, S, Vt = np.linalg.svd(Z, full_matrices=False)

    pc1 = U[:, 0] * S[0]
    loadings = Vt[0]

    # Flip sign if needed so that higher scores = higher overall knowledge
    if np.nansum(loadings) < 0:
        pc1 = -pc1
        loadings = -loadings

    var_ratio = float((S[0] ** 2) / (S**2).sum())

    scores = pd.Series(index=df_in.index, dtype=float)
    scores.loc[mask] = pc1

    meta = {"explained_variance_ratio": var_ratio, "loadings": dict(zip(cols, loadings))}
    return scores, meta


## Preprocessing and derived variables
We derive (1) food neophobia score (X), (2) gardening experience score (W), and the outcomes: (3) knowledge (Y for H1) and (4) consumption (Y for H2).

In [4]:
df = df_raw.copy()

# --- W: Garden experience score (same logic as Analysis 1) ---
df["gardener_binary"] = df["gardener"].replace({"Sim": 1, "Não": 0}).astype(float)

# Decision: years_experience outside [0, 50] => 0
years = pd.to_numeric(df["years_experience"], errors="coerce").fillna(0)
years = years.where((years >= 0) & (years <= 50), 0)
df["years_experience_clean"] = years

freq = pd.to_numeric(df["frequency_experience"], errors="coerce").fillna(0)
df["frequency_experience_clean"] = freq

df["garden_experience_raw"] = (df["gardener_binary"] * (df["years_experience_clean"] * df["frequency_experience_clean"])).astype(float)
df["garden_experience_score"] = safe_normalize_0_1(df["garden_experience_raw"]).astype(float)

df["experience_bin"], experience_bin_ranges = make_equal_frequency_bins_with_zero_as_none(df["garden_experience_score"], n_bins_nonzero=4)

print("Garden experience score summary:")
display(df["garden_experience_score"].describe())
print("Experience bin counts:")
display(df["experience_bin"].value_counts(dropna=False))


Garden experience score summary:


count    157.000000
mean       0.047492
std        0.121287
min        0.000000
25%        0.000000
50%        0.000000
75%        0.037500
max        1.000000
Name: garden_experience_score, dtype: float64

Experience bin counts:


experience_bin
none     92
bin_1    17
bin_4    16
bin_2    16
bin_3    16
Name: count, dtype: int64

In [5]:
# --- X: Food neophobia score (10–70) (same logic as Analysis 1) ---
neophilic_items = ["g06q39", "g06q42", "g06q44", "g06q47", "g06q48"]
neophobic_items = ["g06q40", "g06q41", "g06q43", "g06q45", "g06q46"]

for col in neophilic_items + neophobic_items:
    df[f"{col}_num"] = parse_likert_1_to_7(df[col])

for col in neophilic_items:
    df[f"{col}_rev"] = reverse_1_to_7(df[f"{col}_num"])

df["food_neophobia_score"] = df[[f"{c}_rev" for c in neophilic_items]].sum(axis=1) + df[[f"{c}_num" for c in neophobic_items]].sum(axis=1)

print("Food neophobia score summary (expected 10–70):")
display(df["food_neophobia_score"].describe())


Food neophobia score summary (expected 10–70):


count    157.000000
mean      31.713376
std       10.391103
min       11.000000
25%       24.000000
50%       32.000000
75%       39.000000
max       61.000000
Name: food_neophobia_score, dtype: float64

In [6]:
# --- Y (H1): Knowledge composite via PCA(1) ---
knowledge_components = ["recognition_grade", "classification_grade", "species_total"]
df["knowledge_index_pca"], knowledge_pca_meta = pca_1_component_index(df, knowledge_components)

print("Knowledge PCA meta:")
display(pd.DataFrame({"metric": knowledge_pca_meta.keys(), "value": list(knowledge_pca_meta.values())}))

print("Knowledge index summary (PCA PC1 units):")
display(df["knowledge_index_pca"].describe())

# Rescaled version (0–100) for mock-style plots and interpretability
df["knowledge_index_0_100"] = minmax_scale_0_100(df["knowledge_index_pca"])


Knowledge PCA meta:


,metric,value
0,explained_variance_ratio,0.807022
1,loadings,"{'recognition_grade': 0.6075587555335993, 'cla..."


Knowledge index summary (PCA PC1 units):


count    1.570000e+02
mean     9.051500e-17
std      1.560957e+00
min     -2.810545e+00
25%     -1.226565e+00
50%     -2.629045e-01
75%      1.161364e+00
max      3.816436e+00
Name: knowledge_index_pca, dtype: float64

In [7]:
# --- Y (H2): Consumption index (annualized + log1p + normalized, averaged across groups) ---
for group in ["plants", "mushrooms", "algae"]:
    ann = annualize_consumption(df[f"consumption_{group}_time_scale"], df[f"consumption_{group}_frequency"])
    df[f"consumption_{group}_per_year"] = ann
    df[f"consumption_{group}_per_year_log"] = np.log1p(ann)
    df[f"consumption_{group}_per_year_log_norm"] = safe_normalize_0_1(df[f"consumption_{group}_per_year_log"])

df["consumption_index"] = df[
    [
        "consumption_plants_per_year_log_norm",
        "consumption_mushrooms_per_year_log_norm",
        "consumption_algae_per_year_log_norm",
    ]
].mean(axis=1)

df["consumption_total_per_year"] = df[["consumption_plants_per_year", "consumption_mushrooms_per_year", "consumption_algae_per_year"]].sum(axis=1)

print("Consumption index summary (0–1):")
display(df["consumption_index"].describe())

# Rescaled version (0–100) for mock-style plots
df["consumption_index_0_100"] = minmax_scale_0_100(df["consumption_index"])


Consumption index summary (0–1):


count    157.000000
mean       0.247262
std        0.203238
min        0.000000
25%        0.066019
50%        0.208897
75%        0.396985
max        0.802657
Name: consumption_index, dtype: float64

## Distributions
We visualize the main derived variables and save figures to `data/final_outputs/new_analysis_2/`.

In [8]:
fig_hist = px.histogram(
    df,
    x="garden_experience_score",
    nbins=30,
    title="Distribution of garden experience score (0–1)",
    labels={"garden_experience_score": "Garden experience score (normalized)"},
    color_discrete_sequence=["seagreen"],
)
fig_hist.update_layout(bargap=0.02, showlegend=False)
fig_hist.show()

fig_hist.write_html(images_path / "experience_score_hist.html")
fig_hist.write_image(images_path / "experience_score_hist.png", width=800, height=500, scale=3)
print("Saved experience_score_hist.html and .png")


Saved experience_score_hist.html and .png


In [9]:
fig_neophobia = px.histogram(
    df,
    x="food_neophobia_score",
    nbins=25,
    title="Distribution of food neophobia score (10–70)",
    labels={"food_neophobia_score": "Food neophobia score (sum of 10 items)"},
    color_discrete_sequence=[px.colors.sequential.Greens[2]],
)
fig_neophobia.update_layout(bargap=0.02, showlegend=False)
fig_neophobia.show()

fig_neophobia.write_html(images_path / "food_neophobia_hist.html")
fig_neophobia.write_image(images_path / "food_neophobia_hist.png", width=800, height=500, scale=3)
print("Saved food_neophobia_hist.html and .png")


Saved food_neophobia_hist.html and .png


In [10]:
fig_knowledge = px.histogram(
    df.dropna(subset=["knowledge_index_pca"]),
    x="knowledge_index_pca",
    nbins=25,
    title="Distribution of knowledge index (PCA PC1)",
    labels={"knowledge_index_pca": "Knowledge index (PC1)"},
    color_discrete_sequence=[px.colors.sequential.Greens[4]],
)
fig_knowledge.update_layout(bargap=0.02, showlegend=False)
fig_knowledge.show()

fig_knowledge.write_html(images_path / "knowledge_index_hist.html")
fig_knowledge.write_image(images_path / "knowledge_index_hist.png", width=800, height=500, scale=3)
print("Saved knowledge_index_hist.html and .png")


Saved knowledge_index_hist.html and .png


In [11]:
fig_cons = px.histogram(
    df,
    x="consumption_index",
    nbins=25,
    title="Distribution of consumption index (0–1)",
    labels={"consumption_index": "Consumption index (0–1)"},
    color_discrete_sequence=[px.colors.sequential.Greens[5]],
)
fig_cons.update_layout(bargap=0.02, showlegend=False)
fig_cons.show()

fig_cons.write_html(images_path / "consumption_index_hist.html")
fig_cons.write_image(images_path / "consumption_index_hist.png", width=800, height=500, scale=3)
print("Saved consumption_index_hist.html and .png")


Saved consumption_index_hist.html and .png


## Moderation models
We standardize X and W (z-scores) for interpretability and to reduce multicollinearity in the interaction term.

For each hypothesis, we report model coefficients (OLS with HC3 robust SE when stable), the interaction interpretation, and interaction plots (real data + model predictions).

In [12]:
# Standardize predictors
df["neophobia_z"] = zscore(df["food_neophobia_score"])
df["experience_z"] = zscore(df["garden_experience_score"])

print("Predictor summaries (z-scored):")
display(df[["neophobia_z", "experience_z"]].describe())


Predictor summaries (z-scored):


,neophobia_z,experience_z
count,1.570000e+02,1.570000e+02
mean,2.262875e-17,-9.051500e-17
std,1.003200e+00,1.003200e+00
min,-1.999755e+00,-3.928214e-01
25%,-7.446812e-01,-3.928214e-01
50%,2.767189e-02,-3.928214e-01
75%,7.034808e-01,-8.264725e-02
max,2.827452e+00,7.878489e+00


In [13]:
# English mappings (same as Analysis 1; used for optional adjusted models)
gender_map = {
    "Mulher cis - pessoa que nasceu com sexo feminino e se identifica com o gênero feminino": "Female",
    "Homem cis - pessoa que nasceu com sexo masculino e se identifica com o gênero masculino": "Male",
    "Pessoa não binária - pessoa que não se identifica estritamente com o gênero masculino ou feminino": "Non-binary",
    "Não sei/prefiro não responder": "Prefer not to answer",
}
df["gender_en"] = df["gender"].map(gender_map).fillna("Other / missing")

diet_map = {
    "Onívora - Possuo uma alimentação diversificada consumindo carne, frango, peixe, verduras, frutas, leite, entre outros alimentos": "Omnivore",
    "Flexitariano - Tenho um cardápio flexível e como carne eventualmente ou, pelo menos, tento reduzir as quantidades": "Flexitarian",
    "Vegetariana - Excluo todos os tipos de carnes, aves, peixes e, posso incluir, ovos, laticínios e seus produtos": "Vegetarian",
    "Vegana - Não me alimento de nenhum produto que contenha carne, ovos, leite, mel ou outros ingredientes derivados de animais": "Vegan",
}
df["diet_type_en"] = df["diet_type"].map(diet_map).fillna("Other / missing")

education_level_map = {
    "Graduação em curso": "Undergraduate (in progress)",
    "Graduação concluída": "Undergraduate (completed)",
    "Mestrado em curso": "MSc (in progress)",
    "Mestrado concluído": "MSc (completed)",
    "Doutorado em curso": "PhD (in progress)",
    "Doutorado concluído": "PhD (completed)",
}
df["education_level_en"] = df["education_level"].map(education_level_map).fillna("Other / missing")


def map_education_field_to_broad(field: str) -> str:
    """Map a free-text education field to a broader category (heuristic).

    Args:
        field: Raw text field.

    Returns:
        Broad category label.
    """
    if pd.isna(field):
        return "Missing"
    f = str(field).strip().lower()

    if any(k in f for k in ["nutri", "medicin", "enferm", "odont", "biomed", "psicolog", "farmac", "gastron"]):
        return "Health Sciences"
    if any(k in f for k in ["ecolog", "biolog", "quim", "ambient", "agronom", "florest", "pesca", "ciências biológicas"]):
        return "Natural & Environmental Sciences"
    if any(k in f for k in ["engenhar", "arquitet", "comput", "sistemas", "tecnolog", "redes", "ti", "química", "desenho industrial"]):
        return "Engineering & Technology"
    if any(k in f for k in ["direito", "hist", "letras", "admin", "turismo", "geograf", "comunic", "sociais", "relig", "serviço social", "servico social", "relações internacionais", "pedagogia", "produção cultural", "contábeis", "teatro"]):
        return "Humanities & Social Sciences"

    return "Other"


df["education_field_broad"] = df["education_field"].apply(map_education_field_to_broad)
df["education_field_broad"].value_counts(dropna=False)


education_field_broad
Health Sciences                     61
Humanities & Social Sciences        43
Engineering & Technology            37
Natural & Environmental Sciences    14
Other                                2
Name: count, dtype: int64

In [14]:
def fit_ols_with_fallback(formula: str, df_in: pd.DataFrame) -> tuple[object | None, str]:
    """Fit OLS with robust HC3; fall back to non-robust if unstable.

    Args:
        formula: Patsy formula for statsmodels.
        df_in: Dataframe.

    Returns:
        (model, covariance_type_label)
    """
    # Attempt 1: Robust HC3
    try:
        res = smf.ols(formula, data=df_in).fit(cov_type="HC3")
        if res.bse.isna().any():
            raise ValueError("Unstable HC3 results (NaN SE).")
        return res, "HC3 (Robust)"
    except Exception as e:
        print(f"Notice: Robust covariance failed/unstable ({e}). Falling back to standard errors.")

    # Attempt 2: Standard errors
    try:
        res = smf.ols(formula, data=df_in).fit(cov_type="nonrobust")
        return res, "Standard (Non-robust)"
    except Exception as e2:
        print(f"Standard model also failed: {e2}")
        return None, "failed"


def simple_slopes_from_model(model, x: str, w: str, interaction: str, w_values: list[float]) -> pd.DataFrame:
    """Compute the conditional slope of X for several values of W.

    For a model: Y = b0 + b1*X + b2*W + b3*(X*W),
    slope dY/dX = b1 + b3*W.

    Args:
        model: Fitted statsmodels results.
        x: Name of X coefficient in params.
        w: Name of W coefficient in params (not required for slope).
        interaction: Name of interaction term coefficient in params.
        w_values: List of W values to compute slopes.

    Returns:
        Dataframe with conditional slopes.
    """
    b1 = model.params.get(x, np.nan)
    b3 = model.params.get(interaction, np.nan)

    rows = []
    for wv in w_values:
        rows.append({"experience_z": wv, "slope_neophobia": float(b1 + b3 * wv)})
    return pd.DataFrame(rows)


def make_real_interaction_plot(
    df_in: pd.DataFrame,
    model,
    outcome_raw_col: str,
    outcome_scaled_col: str,
    title: str,
    y_axis_label: str,
    experience_line_quantiles: tuple[float, float] = (0.1, 0.9),
) -> go.Figure:
    """Create a data-driven moderation plot (scatter + predicted lines).

    This replaces earlier mock-style diagrams with plots that show the actual
    participant data (scatter) plus model-predicted lines at low/high gardening
    experience.

    Args:
        df_in: Input dataframe used for fitting (must include raw and z variables).
        model: Fitted moderation model (uses neophobia_z and experience_z).
        outcome_raw_col: Outcome column in the model scale (e.g., knowledge_index_pca).
        outcome_scaled_col: Outcome scaled to 0–100 for plotting (e.g., knowledge_index_0_100).
        title: Figure title.
        y_axis_label: Y axis label (0–100).
        experience_line_quantiles: Quantiles (low, high) of garden_experience_score for lines.

    Returns:
        Plotly figure.
    """
    plot_df = df_in.dropna(subset=["food_neophobia_score", outcome_scaled_col, "garden_experience_score"]).copy()

    # Scatter uses raw neophobia for interpretability
    fig = px.scatter(
        plot_df,
        x="food_neophobia_score",
        y=outcome_scaled_col,
        color="garden_experience_score",
        color_continuous_scale=px.colors.sequential.Greens,
        opacity=0.55,
        labels={
            "food_neophobia_score": "Food neophobia (10–70)",
            outcome_scaled_col: y_axis_label,
            "garden_experience_score": "Garden experience (0–1)",
        },
        title=title,
    )

    # Build prediction grid (raw X)
    x_min = float(plot_df["food_neophobia_score"].min())
    x_max = float(plot_df["food_neophobia_score"].max())
    x_grid = np.linspace(x_min, x_max, 120)

    # Convert raw X to z using the same reference distribution
    x_mean = float(plot_df["food_neophobia_score"].mean())
    x_std = float(plot_df["food_neophobia_score"].std(ddof=0))
    if x_std == 0:
        x_std = 1.0

    # Experience levels for lines (in raw scale, then convert to z)
    q_low, q_high = experience_line_quantiles
    exp_low_raw = float(plot_df["garden_experience_score"].quantile(q_low))
    exp_high_raw = float(plot_df["garden_experience_score"].quantile(q_high))

    w_mean = float(plot_df["garden_experience_score"].mean())
    w_std = float(plot_df["garden_experience_score"].std(ddof=0))
    if w_std == 0:
        w_std = 1.0

    exp_levels = [
        (f"Low experience (q{int(q_low * 100)})", exp_low_raw, "#9aa3a8", "dash"),
        (f"High experience (q{int(q_high * 100)})", exp_high_raw, "seagreen", "solid"),
    ]

    # Outcome scaling for predicted lines (map model-scale predictions to 0–100 using observed min/max)
    y_obs = pd.to_numeric(plot_df[outcome_raw_col], errors="coerce")
    y_min = float(y_obs.min())
    y_max = float(y_obs.max())

    for label, w_raw, color, dash in exp_levels:
        df_pred = pd.DataFrame(
            {
                "neophobia_z": (x_grid - x_mean) / x_std,
                "experience_z": (w_raw - w_mean) / w_std,
            }
        )
        y_pred = model.predict(df_pred)

        if np.isfinite(y_min) and np.isfinite(y_max) and y_max != y_min:
            y_plot = (y_pred - y_min) / (y_max - y_min) * 100
        else:
            y_plot = y_pred

        fig.add_trace(
            go.Scatter(
                x=x_grid,
                y=y_plot,
                mode="lines",
                name=label,
                line=dict(color=color, width=4, dash=dash),
            )
        )

    fig.update_layout(
        yaxis=dict(range=[0, 105]),
        coloraxis_colorbar=dict(title="Experience"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    return fig


## H1 (Cognitive): Does experience buffer the neophobia → knowledge association?

In [15]:
df_h1 = df.dropna(subset=["knowledge_index_pca", "neophobia_z", "experience_z"]).copy()
print("H1 usable N:", len(df_h1))

formula_h1 = "knowledge_index_pca ~ neophobia_z + experience_z + neophobia_z:experience_z"
m_h1, cov_h1 = fit_ols_with_fallback(formula_h1, df_h1)

if m_h1 is not None:
    print(f"Covariance type: {cov_h1}")
    print("--- H1 moderation model (knowledge) ---")
    display(m_h1.summary())

    b_int = m_h1.params.get("neophobia_z:experience_z", np.nan)
    p_int = m_h1.pvalues.get("neophobia_z:experience_z", np.nan)
    b_x = m_h1.params.get("neophobia_z", np.nan)

    print("\nInteraction term (neophobia × experience):")
    print(f"  β_interaction = {b_int:.4f}, p = {p_int:.4f}")

    if np.isfinite(b_x) and np.isfinite(b_int):
        if b_x < 0 and b_int > 0:
            print("Interpretation: Buffering pattern (negative neophobia effect becomes weaker as experience increases).")
        elif b_x < 0 and b_int < 0:
            print("Interpretation: Amplification pattern (negative neophobia effect becomes stronger as experience increases).")
        else:
            print("Interpretation: Interaction direction depends on main effects; see conditional slopes below.")

    slopes = simple_slopes_from_model(m_h1, x="neophobia_z", w="experience_z", interaction="neophobia_z:experience_z", w_values=[-1, 0, 1])
    print("\nConditional slopes of neophobia on knowledge (dY/dX):")
    display(slopes)


H1 usable N: 157
Covariance type: HC3 (Robust)
--- H1 moderation model (knowledge) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     knowledge_index_pca   R-squared:                       0.227
Model:                             OLS   Adj. R-squared:                  0.212
Method:                  Least Squares   F-statistic:                     17.60
Date:                 Tue, 03 Feb 2026   Prob (F-statistic):           7.21e-10
Time:                         11:13:32   Log-Likelihood:                -271.97
No. Observations:                  157   AIC:                             551.9
Df Residuals:                      153   BIC:                             564.2
Df Model:                            3                                         
Covariance Type:                   HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.0059      0.122      0.049      0.961      -0.234       0.246
neophobia_z                 -0.6787      0.137     -4.942      0.000      -0.948      -0.410
experience_z                 0.2137      0.274      0.780      0.435      -0.323       0.750
neophobia_z:experience_z     0.0432      0.472      0.091      0.927      -0.882       0.968
==============================================================================
Omnibus:                        3.349   Durbin-Watson:                   1.543
Prob(Omnibus):                  0.187   Jarque-Bera (JB):                3.027
Skew:                           0.259   Prob(JB):                        0.220
Kurtosis:                       2.559   Cond. No.                         1.40
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""


Interaction term (neophobia × experience):
  β_interaction = 0.0432, p = 0.9271
Interpretation: Buffering pattern (negative neophobia effect becomes weaker as experience increases).

Conditional slopes of neophobia on knowledge (dY/dX):


,experience_z,slope_neophobia
0,-1,-0.721854
1,0,-0.678694
2,1,-0.635534


In [16]:
if m_h1 is not None:
    fig_h1 = make_real_interaction_plot(
        df_in=df_h1,
        model=m_h1,
        outcome_raw_col="knowledge_index_pca",
        outcome_scaled_col="knowledge_index_0_100",
        title="H1 (Cognitive): Neophobia × gardening experience predicting knowledge",
        y_axis_label="Knowledge (0–100)",
        experience_line_quantiles=(0.1, 0.9),
    )
    fig_h1.show()

    fig_h1.write_html(images_path / "h1_interaction_real.html")
    fig_h1.write_image(images_path / "h1_interaction_real.png", width=900, height=550, scale=3)
    print("Saved h1_interaction_real.html and .png")


Saved h1_interaction_real.html and .png


## H2 (Behavioral): Does experience buffer the neophobia → consumption association?

In [17]:
df_h2 = df.dropna(subset=["consumption_index", "neophobia_z", "experience_z"]).copy()
print("H2 usable N:", len(df_h2))

formula_h2 = "consumption_index ~ neophobia_z + experience_z + neophobia_z:experience_z"
m_h2, cov_h2 = fit_ols_with_fallback(formula_h2, df_h2)

if m_h2 is not None:
    print(f"Covariance type: {cov_h2}")
    print("--- H2 moderation model (consumption) ---")
    display(m_h2.summary())

    b_int = m_h2.params.get("neophobia_z:experience_z", np.nan)
    p_int = m_h2.pvalues.get("neophobia_z:experience_z", np.nan)
    b_x = m_h2.params.get("neophobia_z", np.nan)

    print("\nInteraction term (neophobia × experience):")
    print(f"  β_interaction = {b_int:.4f}, p = {p_int:.4f}")

    if np.isfinite(b_x) and np.isfinite(b_int):
        if b_x < 0 and b_int > 0:
            print("Interpretation: Buffering pattern (negative neophobia effect becomes weaker as experience increases).")
        elif b_x < 0 and b_int < 0:
            print("Interpretation: Amplification pattern (negative neophobia effect becomes stronger as experience increases).")
        else:
            print("Interpretation: Interaction direction depends on main effects; see conditional slopes below.")

    slopes = simple_slopes_from_model(m_h2, x="neophobia_z", w="experience_z", interaction="neophobia_z:experience_z", w_values=[-1, 0, 1])
    print("\nConditional slopes of neophobia on consumption (dY/dX):")
    display(slopes)


H2 usable N: 157
Covariance type: HC3 (Robust)
--- H2 moderation model (consumption) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      consumption_index   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     26.01
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           1.19e-13
Time:                        11:13:34   Log-Likelihood:                 52.927
No. Observations:                 157   AIC:                            -97.85
Df Residuals:                     153   BIC:                            -85.63
Df Model:                           3                                         
Covariance Type:                  HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.2476      0.015     16.748      0.000       0.219       0.277
neophobia_z                 -0.1027      0.015     -6.670      0.000      -0.133      -0.073
experience_z                 0.0146      0.028      0.520      0.603      -0.040       0.069
neophobia_z:experience_z     0.0023      0.052      0.043      0.965      -0.101       0.105
==============================================================================
Omnibus:                        5.884   Durbin-Watson:                   1.916
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                6.013
Skew:                           0.475   Prob(JB):                       0.0495
Kurtosis:                       2.868   Cond. No.                         1.40
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""


Interaction term (neophobia × experience):
  β_interaction = 0.0023, p = 0.9654
Interpretation: Buffering pattern (negative neophobia effect becomes weaker as experience increases).

Conditional slopes of neophobia on consumption (dY/dX):


,experience_z,slope_neophobia
0,-1,-0.104999
1,0,-0.102722
2,1,-0.100444


In [18]:
if m_h2 is not None:
    fig_h2 = make_real_interaction_plot(
        df_in=df_h2,
        model=m_h2,
        outcome_raw_col="consumption_index",
        outcome_scaled_col="consumption_index_0_100",
        title="H2 (Behavioral): Neophobia × gardening experience predicting consumption",
        y_axis_label="Consumption (0–100)",
        experience_line_quantiles=(0.1, 0.9),
    )
    fig_h2.show()

    fig_h2.write_html(images_path / "h2_interaction_real.html")
    fig_h2.write_image(images_path / "h2_interaction_real.png", width=900, height=550, scale=3)
    print("Saved h2_interaction_real.html and .png")


Saved h2_interaction_real.html and .png


## Exploratory appendix: barriers by experience bin
Barriers are not part of H1/H2, but they are relevant to interpret mechanisms (knowledge/access/intention). We report descriptive tables and Cramér’s V effect size.

In [19]:
df_bins = df[df["experience_bin"].isin(["none", "bin_1", "bin_2", "bin_3", "bin_4"])].copy()
df_bins["experience_bin"] = pd.Categorical(df_bins["experience_bin"], categories=["none", "bin_1", "bin_2", "bin_3", "bin_4"], ordered=True)

barrier_cols = ["barrier_plant", "barrier_mushroom", "barrier_algae"]

for col in barrier_cols:
    print("\n" + "=" * 90)
    print(f"Barrier: {col}")
    print("=" * 90)

    tab = pd.crosstab(df_bins["experience_bin"], df_bins[col])
    tab_pct = tab.div(tab.sum(axis=1), axis=0) * 100

    display(tab.astype(int).astype(str) + " (" + tab_pct.round(1).astype(str) + "%)")
    print(f"Cramér's V: {cramers_v_from_crosstab(tab):.3f}")



Barrier: barrier_plant


barrier_plant,Falta de acesso econômico: acho esse tipo de alimento caro,Falta de acesso físico: não é fácil de achar esses alimentos perto de lugares onde eu moro,Falta de conhecimento: não conheço esses alimentos e nem sei como preparar,Não consumo porque não tenho intenção,Outro
experience_bin,,,,,
none,8 (8.7%),25 (27.2%),42 (45.7%),15 (16.3%),2 (2.2%)
bin_1,0 (0.0%),6 (35.3%),9 (52.9%),2 (11.8%),0 (0.0%)
bin_2,0 (0.0%),11 (68.8%),1 (6.2%),1 (6.2%),3 (18.8%)
bin_3,1 (6.2%),10 (62.5%),1 (6.2%),0 (0.0%),4 (25.0%)
bin_4,2 (12.5%),7 (43.8%),5 (31.2%),0 (0.0%),2 (12.5%)


Cramér's V: 0.270

Barrier: barrier_mushroom


barrier_mushroom,Falta de acesso econômico: acho esse tipo de alimento caro,Falta de acesso físico: não é fácil de achar esses alimentos perto de lugares onde eu moro,Falta de conhecimento: não conheço esses alimentos e nem sei como preparar,Não consumo porque não tenho intenção,Outro
experience_bin,,,,,
none,38 (41.3%),15 (16.3%),18 (19.6%),15 (16.3%),6 (6.5%)
bin_1,7 (41.2%),4 (23.5%),3 (17.6%),2 (11.8%),1 (5.9%)
bin_2,5 (31.2%),6 (37.5%),3 (18.8%),1 (6.2%),1 (6.2%)
bin_3,11 (68.8%),3 (18.8%),0 (0.0%),0 (0.0%),2 (12.5%)
bin_4,8 (50.0%),4 (25.0%),1 (6.2%),1 (6.2%),2 (12.5%)


Cramér's V: 0.161

Barrier: barrier_algae


barrier_algae,Falta de acesso econômico: acho esse tipo de alimento caro,Falta de acesso físico: não é fácil de achar esses alimentos perto de lugares onde eu moro,Falta de conhecimento: não conheço esses alimentos e nem sei como preparar,Não consumo porque não tenho intenção,Outro
experience_bin,,,,,
none,19 (20.7%),25 (27.2%),21 (22.8%),21 (22.8%),6 (6.5%)
bin_1,5 (29.4%),5 (29.4%),5 (29.4%),1 (5.9%),1 (5.9%)
bin_2,1 (6.2%),4 (25.0%),9 (56.2%),1 (6.2%),1 (6.2%)
bin_3,6 (37.5%),3 (18.8%),3 (18.8%),2 (12.5%),2 (12.5%)
bin_4,2 (12.5%),8 (50.0%),1 (6.2%),1 (6.2%),4 (25.0%)


Cramér's V: 0.213


---

# Results Summary Report

## Overview

This analysis examined whether **gardening experience** buffers (moderates) the negative relationship between **food neophobia** and two outcomes: (1) **knowledge** and (2) **consumption** of neglected/unconventional foods (plants, mushrooms, algae).

### Sample Characteristics
- **Total N = 157** participants with complete data
- **Gardening experience**: Highly skewed distribution; 58.6% of participants have no gardening experience (n=92), while 41.4% have some experience (n=65 across 4 bins)
- **Food neophobia**: Mean = 31.7 (SD = 10.4), range 11–61 on a 10–70 scale — indicating moderate neophobia levels overall
- **Knowledge index**: PCA-based composite (PC1 explains **80.7%** of variance in recognition, classification, and species recall)
- **Consumption index**: Annualized, log-transformed, normalized average across plants/mushrooms/algae (Mean = 0.25, SD = 0.20)

---

## Hypothesis Testing Results

### H1 (Cognitive): Does gardening experience buffer the neophobia → knowledge association?

| Predictor | β | SE | p-value | Interpretation |
|-----------|---|----|---------| --------------|
| Intercept | 0.006 | 0.122 | .961 | — |
| **Neophobia (z)** | **−0.679** | 0.137 | **< .001*** | Strong negative effect |
| Experience (z) | 0.214 | 0.274 | .435 | Non-significant |
| **Neophobia × Experience** | **0.043** | 0.472 | **.927** | **Not significant** |

**Model fit**: R² = 0.227 (Adj. R² = 0.212), F(3, 153) = 17.60, p < .001

#### Interpretation
- **Food neophobia has an interesting negative effect on knowledge**: For each 1 SD increase in neophobia, knowledge decreases by ~0.68 SD units (p < .001).
- **The interaction term is not significant** (β = 0.04, p = .93): Gardening experience does **not** buffer the negative neophobia–knowledge relationship.
- Conditional slopes are virtually identical at low (−0.72), mean (−0.68), and high (−0.64) experience levels.

> ⚠️ **H1 is NOT supported**: The buffering hypothesis for the cognitive outcome was not confirmed.

---

### H2 (Behavioral): Does gardening experience buffer the neophobia → consumption association?

| Predictor | β | SE | p-value | Interpretation |
|-----------|---|----|---------| --------------|
| Intercept | 0.248 | 0.015 | < .001 | — |
| **Neophobia (z)** | **−0.103** | 0.015 | **< .001*** | Significant negative effect |
| Experience (z) | 0.015 | 0.028 | .603 | Non-significant |
| **Neophobia × Experience** | **0.002** | 0.052 | **.965** | **Not significant** |

**Model fit**: R² = 0.273 (Adj. R² = 0.259), F(3, 153) = 26.01, p < .001

#### Interpretation
- **Food neophobia has a significant negative effect on consumption**: For each 1 SD increase in neophobia, consumption decreases by ~0.10 units on the 0–1 scale (p < .001).
- **The interaction term is essentially zero** (β = 0.002, p = .97): Gardening experience does **not** moderate the neophobia–consumption relationship.
- Conditional slopes are nearly identical across experience levels (−0.10 at all levels).

> ⚠️ **H2 is NOT supported**: The buffering hypothesis for the behavioral outcome was not confirmed.

---

## Key Findings Summary

| Hypothesis | Main Effect (Neophobia) | Interaction (Buffering) | Conclusion |
|------------|------------------------|------------------------|------------|
| H1 (Knowledge) | β = −0.68, p < .001 ✓ | β = 0.04, p = .93 ✗ | **Not supported** |
| H2 (Consumption) | β = −0.10, p < .001 ✓ | β = 0.002, p = .97 ✗ | **Not supported** |

### Main Conclusions

1. **Food neophobia consistently predicts lower knowledge and consumption** of neglected foods, confirming its role as a psychological barrier.

2. **Gardening experience does NOT buffer** these negative effects. The interaction terms are essentially zero (p > .92 for both models).

3. **Experience itself shows no significant direct effect** on knowledge (p = .44) or consumption (p = .60) when controlling for neophobia.

4. **Model explanatory power is moderate**: R² = 22.7% for knowledge and 27.3% for consumption, suggesting other unmeasured factors also play important roles.

---

## Exploratory Findings: Barriers to Consumption by Experience Level

The barrier analysis (Cramér's V effect sizes) suggests interesting patterns:

| Barrier Type | Cramér's V | Pattern |
|-------------|-----------|---------|
| Plants | 0.270 | Moderate association |
| Mushrooms | 0.161 | Small association |
| Algae | 0.213 | Small-moderate association |

**Observations**:
- Among non-gardeners, **"lack of knowledge"** is the most cited barrier for plants (45.7%)
- Among experienced gardeners (bins 2–4), **"lack of physical access"** becomes the primary barrier for plants (62–69%)
- This shift suggests experienced gardeners have overcome knowledge barriers but face access constraints


In [20]:
def summarize_moderation(model, label: str) -> None:
    """Print a compact moderation summary.

    Args:
        model: statsmodels fitted results (or None).
        label: Label for the hypothesis.
    """
    print("\n" + "#" * 100)
    print(label)
    print("#" * 100)

    if model is None:
        print("Model not available.")
        return

    b_x = model.params.get("neophobia_z", np.nan)
    p_x = model.pvalues.get("neophobia_z", np.nan)
    b_w = model.params.get("experience_z", np.nan)
    p_w = model.pvalues.get("experience_z", np.nan)
    b_int = model.params.get("neophobia_z:experience_z", np.nan)
    p_int = model.pvalues.get("neophobia_z:experience_z", np.nan)

    print(f"Neophobia main effect: β = {b_x:.4f}, p = {p_x:.4f}")
    print(f"Experience main effect: β = {b_w:.4f}, p = {p_w:.4f}")
    print(f"Interaction (buffering) : β = {b_int:.4f}, p = {p_int:.4f}")

    slopes = simple_slopes_from_model(model, x="neophobia_z", w="experience_z", interaction="neophobia_z:experience_z", w_values=[-1, 0, 1])
    print("Conditional slopes (dY/dX) at W=-1,0,+1:")
    display(slopes)


summarize_moderation(m_h1, "H1 (Knowledge outcome)")
summarize_moderation(m_h2, "H2 (Consumption outcome)")



####################################################################################################
H1 (Knowledge outcome)
####################################################################################################
Neophobia main effect: β = -0.6787, p = 0.0000
Experience main effect: β = 0.2137, p = 0.4351
Interaction (buffering) : β = 0.0432, p = 0.9271
Conditional slopes (dY/dX) at W=-1,0,+1:


,experience_z,slope_neophobia
0,-1,-0.721854
1,0,-0.678694
2,1,-0.635534



####################################################################################################
H2 (Consumption outcome)
####################################################################################################
Neophobia main effect: β = -0.1027, p = 0.0000
Experience main effect: β = 0.0146, p = 0.6028
Interaction (buffering) : β = 0.0023, p = 0.9654
Conditional slopes (dY/dX) at W=-1,0,+1:


,experience_z,slope_neophobia
0,-1,-0.104999
1,0,-0.102722
2,1,-0.100444
